# Census Dash (base project)

## Import Libraries

In [1]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

## Import Data into DataFrames

### "Population & Single Race"

In [2]:
df_dict = {} # create empty dict to append looped lists to
for year in [2000, 2010, 2020]: # loop the list of years for the data
    file_name = f"{year}_county_census_data.xlsx" # use variable to grab correct .xlsx file
    sheet_name = "Population & Single Race" # pull the proper sheet
    df = pd.read_excel(file_name, sheet_name,dtype = {'State': str, 'County': str, 'FIPS': str}) # keep these columns as strings for the mapping
    df_dict[year] = df # everything into an dict so it can be pulled by index

### "Population & Single Race - %"

In [3]:
percent_dict = {}
for year in [2000, 2010, 2020]:
    file_name = f"{year}_county_census_data.xlsx"
    sheet_name = "Population & Single Race - %"
    percent_df = pd.read_excel(file_name, sheet_name, dtype = {'State': str, 'County': str, 'FIPS': str})
    percent_dict[year] = percent_df

## Import GEOJSON

In [4]:
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

## Build out app

There are three maps, manipulated by a slider, that has to count for two visualizations, right?

In [5]:
# define the app variable to update layout, callbacks, and eventually run
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

year_marks = {2000: '2000', 2010: '2010', 2020: '2020'} # Dict for slider marks

app.layout = html.Div([
    dcc.Slider(
        id='year-slider',  #id for callbacks
        min=2000,
        max=2020,
        step=10,
        marks=year_marks, #slider marks to call data per year based on slider location
        value=2000 #slider handle location when app starts or reloads, referenced by callback decorator and callback function for interoperability
    ),

    dcc.Graph(id='map-graph') # reference to the app call back component id for output, will display that component id's property
])

# Define the callback function this allows the slider handle location to update the map
# In Dash, anytime two components need to be connected to work with eachother, they need to use callback for interactivity

@app.callback( # call back decorator
    Output(component_id='map-graph', component_property='figure'),
    [Input(component_id='year-slider', component_property='value')]) #value is what the user selected with the slider handle location

def update_output(value): # call back function, 'value' refers to the value of the component property in the callback above
    df = df_dict[value]
    percent_df = percent_dict[value]

    fig_map = px.choropleth_mapbox(df, geojson=counties, locations='FIPS', color='Total',
                               color_continuous_scale="Viridis",
                               range_color=(0, 100000),
                               mapbox_style="carto-positron",
                               zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                               opacity=0.5,
                               labels={'Total':'Population'})
    fig_map.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

    fig_map.update_traces(customdata = percent_df, hovertemplate='<br>'.join([ #percent_df coming from percent_dict
    'Name: %{customdata[0]}',
    'Total Population: %{customdata[1]:,}',
    '% White: %{customdata[5]:.2f}%',
    '% Black or African: %{customdata[6]:.2f}%',
    '% Hispanic or Latino: %{customdata[7]:.2f}%',
    '% American Indian and Alaska Native: %{customdata[8]:.2f}%',
    '% Asian: %{customdata[9]:.2f}%',
    '% Native Hawaiian and Other Pacific Islander: %{customdata[10]:.2f}%',
    '% Some Other Race: %{customdata[11]:.2f}%'
    ]))
    return fig_map # assigns the returned to the component property of the app callback output 

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False) # must have use_reloader as False for .ipynb

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
